<a href="https://colab.research.google.com/github/mark-bell-tna/ComputationalAccess/blob/main/Discovery_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os

In [2]:
# Where do you want to get data from - Google Drive or Github
data_source = "Github"  # Change to either Github or Drive - if Drive, copy the data into a folder called "Data" within your "My Drive folder"

In [3]:
if data_source == "Github":
    !git clone https://github.com/mark-bell-tna/ComputationalAccess.git
    sys.path.insert(0, 'ComputationalAccess')
    data_folder = "./"
    os.listdir(data_folder)
else:
    # Connect to gdrive
    from google.colab import drive
    drive.mount('/content/gdrive')
    data_folder = "/content/gdrive/My Drive/Data/"
    os.listdir(data_folder)

Cloning into 'ComputationalAccess'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 157 (delta 21), reused 0 (delta 0), pack-reused 112
Receiving objects: 100% (157/157), 329.78 KiB | 1.35 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [11]:
from disco_search import DiscoView
from ukgwa_textindex import UKGWATextIndex
from operator import itemgetter

Select a random sample of records from the catalogue hierarchy.

Sampling starts at series id(s) input into sample_leaves procedure, and occurs at each level of the hierarchy on the way down. This is because while many series only have a few sub-series, some can have over 200.

min_sample parameter is the minimum size of a sample, as 0.01 percent of a handful is very small (i.e. 0 after rounding)
sample_pct parameter is mainly aimed at sampling the bulk of the records at the bottom of the tree but does have some influence at the top of the catalogue where there are c.450 series in total.

All the print statements will be removed eventually :-)
    

In [13]:
D = DiscoView(page_limit=500)  # page limit parameter is to do with the Discovery API, no advantage in changing it

D.min_sample=10   # Increase to 20 for War Diaries
D.sample_pct=0.03  # Maybe up this to 0.05 too
#D._sample_leaves('C188', 'C142','C201','C287')  # Uncomment to sample from Iraq, HCA, Land Revenue, Exchequer Maps (all randomly chosen)
D._sample_leaves('A13530124')   # Uncomment to sample from whole catalogue
#D._sample_leaves('C14303')  # Uncomment to sample from WO 95

Crawl: ['A13530124']
Sample: 10 Retrieved: 0
Id: A13530124 Count: 452 Total: 0
Sample: 1 Retrieved: 0
Id: C426 Count: 1 Total: 452
Sample: 0 Retrieved: 0
Id: C1590 Count: 13 Total: 453
Sample: 5 Retrieved: 0
Id: C90 Count: 5 Total: 466
Sample: 6 Retrieved: 0
Id: C1270 Count: 6 Total: 471
Sample: 0 Retrieved: 0
Id: C6453 Count: 16 Total: 477
Sample: 5 Retrieved: 0
Id: C6451 Count: 5 Total: 493
Sample: 3 Retrieved: 0
Id: C22776 Count: 3 Total: 498
Sample: 3 Retrieved: 0
Id: C22775 Count: 3 Total: 501
Sample: 0 Retrieved: 0
Id: C22774 Count: 29 Total: 504
Sample: 0 Retrieved: 0
Id: C6450 Count: 16 Total: 533
Sample: 3 Retrieved: 0
Id: C6448 Count: 3 Total: 549
Sample: 0 Retrieved: 0
Id: C71250 Count: 20 Total: 552
Sample: 3 Retrieved: 0
Id: C71249 Count: 3 Total: 572
Sample: 1 Retrieved: 0
Id: C71248 Count: 65 Total: 575
Sample: 3 Retrieved: 0
Id: C6446 Count: 3 Total: 640
Sample: 3 Retrieved: 0
Id: C22766 Count: 3 Total: 643
Sample: 0 Retrieved: 0
Id: C22765 Count: 13 Total: 646
Sample: 

Index the text from the catalogue descriptions for later use (e.g. keyword/phrase search)

In [14]:
T = UKGWATextIndex(stop_words = ["", "and", "of", "the", "in", "a", "by", "which", "their","as","an",
                                 "for","to","if","be","this","on","are","at","were","it","is","that",
                                 "from","been","has","have","or","there","was","they","with","these"])

for idx in D:
    desc = D.get_field(idx, "Description")
    if desc is None:
        continue
    T.add_tokens(D.get_field(idx, "Description").split(" "), idx)
 

Get top N phrases from the indexed text

In [17]:
N = 20  # Return top 20 results (by count)

P = T.get_phrases(min_count = 5, min_length = 2)  # Phrases occurring at least 5 times with at least 2 words in them
P.sort(key=itemgetter(1), reverse=True)
for p in P[:N]:
    print(p)

[['general', 'nursing', 'council'], 20]
[['general', 'nursing', 'council', 'england', 'wales'], 13]
[['correspondence', 'papers'], 12]
[['details', 'piece', 'shown', 'items', 'level'], 11]
[['registered', 'files'], 10]
[['nurses', 'rules', 'revised'], 9]
[['works', 'art'], 9]
[['training', 'schools'], 7]
[['national', 'health', 'service'], 6]
[['please', 'see'], 6]
[['websites', 'please'], 6]
[['its', 'committees'], 6]
[['records', 'relating'], 6]
[['papers', 'relating'], 6]
[['scale', '1', 'inch'], 5]
[['nurse', 'training', 'committees'], 5]
[['division', 'registered', 'files'], 5]
[['compass', 'indicator'], 5]
[['reference', 'table'], 5]
[['female', 'instructions'], 5]
